In [51]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
import ngsolve
from xfem import *
from ngsolve import specialcf

In [52]:
maxh = 0.1
# Finite element space order
order = 1
# Stabilization parameter for ghost-penalty
gamma_stab = 0.1
# Stabilization parameter for Nitsche
lambda_nitsche = 10 * order * order

In [53]:
square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [54]:
r = sqrt(x**2 + y**2)
levelset = r-1

exact = sin(x+y)
coeff_f = - (exact.Diff(x).Diff(x) + exact.Diff(y).Diff(y)).Compile()

In [55]:


maxh = 0.2
h = specialcf.mesh_size
#hf = get_length_of_edge(mesh, mesh.edges[0])
n = Normalize(grad(lsetp1))

g = CoefficientFunction(0)

square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)
         # Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
         #     ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
neg = ci.GetElementsOfType(NEG)
hasif = ci.GetElementsOfType(IF)
haspos = ci.GetElementsOfType(HASPOS)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
patch_elements = ci.GetElementsOfType(ANY)

from xfem import XFESpace

# ... mesh and lsetp1 definition as before ...

Vh = VectorH1(mesh, order=2, dirichlet=[], dgjumps=True)
Qh = H1(mesh, order=1, dirichlet=[], dgjumps=True)
X = Vh * Qh
gfu = GridFunction(X)
(u, p), (v, q) = X.TnT()

# Now you can use cut integration domains:
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)
dw = dFacetPatch(definedonelements=ba_facets)

# ... rest of your variational formulation ...



gamma_stab = 0.01
beta_1 = 0.1
beta_2 = 0.1
beta_3 = 0.1

stokes = InnerProduct(Grad(u), Grad(v)) * dx - div(u)*q*dx - div(v)*p*dx + gamma_stab / h * u * v * ds
stokes += beta_2 * h**2 * (u - u.Other()) * (v - v.Other()) * dw #hf
stokes += -(Grad(u)*n * v + Grad(v)*n * u) * ds
stokes += n*v* p * ds + n*u*q * ds 
stokes += beta_3 *h**3 *(Grad(p)*n-Grad(p.Other())*n)*(Grad(q)*n-Grad(q.Other())*n) * dw #hf
a = BilinearForm(stokes).Assemble()

f = LinearForm(X)
f_rhs = CoefficientFunction((coeff_f, coeff_f))  # or your desired vector-valued RHS
f += InnerProduct(f_rhs, v) * dx

# For the velocity test function v (vector-valued)
f += InnerProduct(g * (gamma_stab / h * v + Grad(v)*n), n) * ds # InnerProduct not needed, as g is scalar

# For the pressure test function q (scalar)
f += g * q * ds # This is a vector, but q*n is only valid if you integrate against a vector-valued test function

#f += g * (gamma_stab / h * v + Grad(v)*n + q*n) * ds
f += - beta_1 * h * f_rhs* Grad(q) * dx
f.Assemble()
# Solve linear system
gfu.vec.data = a.mat.Inverse(X.FreeDofs(),inverse="sparsecholesky") * f.vec
Draw(mesh, lsetp1, "levelset")
Draw(gfu.components[0], mesh, "velocity")
#l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
#print("L2 Error: {0}".format(l2error), "meshzise: {0}".format(maxh), "lambda_nitsche: {0}".format(lambda_nitsche), "gamma_stab: {0}".format(gamma_stab))

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [64]:
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)
dw = dFacetPatch(definedonelements=ba_facets)
dxbar = dCut(lsetp1, HASNEG, definedonelements=hasneg)

NgException: lsets need to be GridFunctions!

In [63]:
dxbar = dx(definedonelements=ci.GetElementsOfType(HASNEG))
dxinner = dx(definedonelements=ci.GetElementsOfType(NEG))
dxcut = dx(definedonelements=ci.GetElementsOfType(IF))
ds = dCut(lsetp1, IF, definedonelements=hasif)
deb = dx(definedonelements=hasany, element_boundary=True)

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: xfem.xfem.CutDifferentialSymbol, levelset_domain: dict, definedon: Optional[Union[ngsolve.comp.Region, str]] = None, vb: ngsolve.comp.VorB = <VorB.VOL: 0>, element_boundary: bool = False, element_vb: ngsolve.comp.VorB = <VorB.VOL: 0>, skeleton: bool = False, deformation: ngsolve.comp.GridFunction = None, definedonelements: pyngcore.pyngcore.BitArray = None) -> xfem.xfem.CutDifferentialSymbol

Invoked with: <xfem.xfem.CutDifferentialSymbol object at 0x000002F2337E4670>; kwargs: definedonelements=<pyngcore.pyngcore.BitArray object at 0x000002F272F716F0>

In [67]:
from xfem import HASNEG

maxh = 0.2
order = 2
h = specialcf.mesh_size
#hf = get_length_of_edge(mesh, mesh.edges[0])
n = Normalize(grad(lsetp1))

g = CoefficientFunction((0, 0))

square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)
         # Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
         #     ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
neg = ci.GetElementsOfType(NEG)
hasif = ci.GetElementsOfType(IF)
haspos = ci.GetElementsOfType(HASPOS)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
patch_elements = ci.GetElementsOfType(ANY)

# ... mesh and lsetp1 definition as before ...

Vh = VectorH1(mesh, order=order, dirichlet=[], dgjumps=True)
Qh = H1(mesh, order=1, dirichlet=[], dgjumps=True)
X = Vh * Qh
gfu = GridFunction(X)
(u, p), (v, q) = X.TnT()

# Now you can use cut integration domains:
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)
dw = dFacetPatch(definedonelements=ba_facets)
dxbar = dCut(lsetp1, HASNEG, definedonelements=hasneg)

# ... rest of your variational formulation ...



gamma_stab = 10 * order * order
beta_0 = 0.1
beta_1 = 0.1
beta_2 = 0.5
beta_3 = 0.5

stokes = InnerProduct(Grad(u), Grad(v)) * dx - div(u)*q*dx - div(v)*p*dx + gamma_stab / h * u * v * ds #a,b
stokes += beta_2 * h * (Grad(u)*n - Grad(u.Other())*n) * (Grad(v)*n - Grad(v.Other())*n) * dw #i #hf
stokes += -(Grad(u)*n * v + Grad(v)*n * u) * ds #a
stokes += n*v* p * ds + n*u*q * ds #b
stokes += beta_3 *h**3 *(Grad(p)*n-Grad(p.Other())*n)*(Grad(q)*n-Grad(q.Other())*n) * dw #j #hf
stokes += -beta_0 * h* (p - p.Other()) * (q - q.Other()) * dw #c #hf
stokes += -beta_1 * h**2 * Grad(p)  * Grad(q) * dx #c #hf
a = BilinearForm(stokes).Assemble()

f = LinearForm(X)
f_rhs = CoefficientFunction((1, 0))
#f_rhs = CoefficientFunction((coeff_f, coeff_f))  # or your desired vector-valued RHS
f += InnerProduct(f_rhs, v) * dx
f += g * (gamma_stab / h * v + Grad(v)*n + q*n) * ds
f += - beta_1 * h**2 * f_rhs* Grad(q) * dx
f.Assemble()

# Solve linear system
gfu.vec.data = a.mat.Inverse(X.FreeDofs(),inverse="sparsecholesky") * f.vec
Draw(mesh, lsetp1, "levelset")
Draw(gfu.components[0], mesh, "velocity")
#l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
#print("L2 Error: {0}".format(l2error), "meshzise: {0}".format(maxh), "lambda_nitsche: {0}".format(lambda_nitsche), "gamma_stab: {0}".format(gamma_stab))

NgException: lsets need to be GridFunctions!

In [ ]:
maxh = 0.4
h = specialcf.mesh_size
#hf = get_length_of_edge(mesh, mesh.edges[0])
n = Normalize(grad(lsetp1))

g = CoefficientFunction((0, 0))

square = SplineGeometry()
square.AddRectangle((-1.25, -1.25), (1.25, 1.25), bc=1)
ngmesh = square.GenerateMesh(maxh=maxh)
mesh = Mesh(ngmesh)
lsetp1 = GridFunction(H1(mesh))
InterpolateToP1(levelset,lsetp1)
         # Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
         #     ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
neg = ci.GetElementsOfType(NEG)
hasif = ci.GetElementsOfType(IF)
haspos = ci.GetElementsOfType(HASPOS)
ba_facets = GetFacetsWithNeighborTypes(mesh, a=haspos, b=any)
patch_elements = ci.GetElementsOfType(ANY)

# ... mesh and lsetp1 definition as before ...

Vh = VectorH1(mesh, order=2, dirichlet=[], dgjumps=True)
Qh = H1(mesh, order=0, dirichlet=[], dgjumps=True)
X = Vh * Qh
gfu = GridFunction(X)
(u, p), (v, q) = X.TnT()

# Now you can use cut integration domains:
dx = dCut(lsetp1, NEG, definedonelements=hasneg)
ds = dCut(lsetp1, IF, definedonelements=hasif)
dw = dFacetPatch(definedonelements=ba_facets)

# ... rest of your variational formulation ...



gamma_stab = 10 * order * order
beta_1 = 0.0
beta_2 = 0.1
beta_0 = 0.1

stokes = InnerProduct(Grad(u), Grad(v)) * dx - div(u)*q*dx - div(v)*p*dx + gamma_stab / h * u * v * ds
stokes += beta_2 * h**2 * (Grad(u) - Grad(u.Other())) * (Grad(v) - Grad(v.Other())) * dw #hf
stokes += -(Grad(u)*n * v + Grad(v)*n * u) * ds
stokes += n*v* p * ds + n*u*q * ds 
stokes += beta_0 *h *(p-p.Other())*(q-q.Other()) * dw #hf
a = BilinearForm(stokes).Assemble()

f = LinearForm(X)
f_rhs = CoefficientFunction((1, 0))
#f_rhs = CoefficientFunction((coeff_f, coeff_f))  # or your desired vector-valued RHS
f += InnerProduct(f_rhs, v) * dx
f += g * (gamma_stab / h * v + Grad(v)*n + q*n) * ds
#f += - beta_1 * h**2 * f_rhs* Grad(q) * dx
f.Assemble()

# Solve linear system
gfu.vec.data = a.mat.Inverse(X.FreeDofs(),inverse="sparsecholesky") * f.vec
Draw(mesh, lsetp1, "levelset")
Draw(gfu.components[0], mesh, "velocity")
#l2error.append(sqrt(Integrate((gfu - exact)**2*dx, mesh)))
#print("L2 Error: {0}".format(l2error), "meshzise: {0}".format(maxh), "lambda_nitsche: {0}".format(lambda_nitsche), "gamma_stab: {0}".format(gamma_stab))

NgException: SymblicBFI needs scalar-valued CoefficientFunction